# Creating the Drude model
##### by Justin Chen and Collen Moe

## About
I wanted to create a model in which reflects the druid model

## Druid model
The Drude model of electrical conduction was proposed in 1900 by Paul Drude to explain the transport properties of electrons in materials (especially metals). Basically, Ohm's law was well established and stated that the current J and voltage V driving the current are related to the resistance R of the material.

# the start / idea.

Used the Ideal gas code as the base of the coding

Suppose we wanted to model the transportation of electons through a system thats where the drude model is used.


## Start of the Drude Model

In [1]:
from vpython import *
import numpy as np
import random as rand
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [2]:
def check_collisions():
    """Determine all pairs of colliding spheres in the global particles[] list. 
        
    """
    collided_pairs = [] #a list of pairs of spheres that are overlapping
    
    for i in range(N):
        for j in range(i+1,N): #start at i+1 to avoid double counting and to avoid self-collision
            d = mag(particles[i].pos - particles[j].pos)
            if d < 2*R:
                collided_pairs.append([i,j])            
    
    return collided_pairs

In [3]:
def calc_collision(i,j):
    """Calculate new velocity, momentum, and position of pairs of colliding spheres in the global particles[] list. 

    Keyword arguments:
    i -- index of one colliding sphere
    j -- index of second colliding sphere

    """

    global particles
    
    r1 = particles[i].pos
    r2 = particles[j].pos
    v1 = particles[i].v
    v2 = particles[j].v
    p1 = m*v1
    p2 = m*v2
    v1i = v1
    v2i = v2
    
    #transform to reference frame of particle 1
    v2rel = v2 - v1
    r2rel = r2 - r1
    
    #find time when collision ocurred
    dx = dot(r2rel, hat(v2rel))
    dy = mag(cross(r2rel, hat(v2rel)))
    alpha = asin(dy/2/R)
    d = 2*R*cos(alpha) + dx
    deltat = d/mag(v2rel)
    
    #move particles backward in time to their locations at the collision
    particles[i].pos = particles[i].pos - particles[i].v*deltat
    particles[j].pos = particles[j].pos - particles[j].v*deltat
    
    #calculate new momenta using CM reference frame
    r1 = particles[i].pos
    r2 = particles[j].pos
    M = 2*m #total mass
    ptot = p1+p2 #total momentum
    vcm = ptot/M
    v1rel = v1 - vcm
    v2rel = v2 - vcm
    p1rel = m*v1rel
    p2rel = m*v2rel
    r = r2 - r1
    p1rel = p1rel - 2*dot(p1rel,hat(r))*hat(r)
    p2rel = p2rel - 2*dot(p2rel,hat(r))*hat(r)
    v1rel = p1rel/m
    v2rel = p2rel/m
    v1 = vcm + v1rel
    v2 = vcm + v2rel
    p1 = m*v1
    p2 = m*v2
    particles[i].v = v1
    particles[j].v = v2

    #move particles forward in time to their locations at the end of the time step
    particles[i].pos = particles[i].pos + particles[i].v*deltat
    particles[j].pos = particles[j].pos + particles[j].v*deltat
    

## Updating the location of the particles/electron
with electrons going through the system we want to make is so that when they reach the end which is the right wall it will teleport to the left to show that more electrons are going through and going around the system same is also applied when it reaches the left wall it will teleport to the right sinces it will be colloding with the atoms and other electron particles its possible for them to make it back up to the system.

In [4]:
scene = canvas(title="N Particles in a Box")

#constants
m = 1.7e-27 #mass of atom in kg
R = 0.5e-10 #radius of atom in m
L = 40*R #length of box in m
thick = L/100 #thickness of box wall in m
k = 1.4e-23 #boltzmann constant
T = 300 #temp in K
s =  np.sqrt(2*3/2*k*T/m) #initial speed


Lwall = box(pos = vec(-2*L, 0, 0), size = vec(thick, L, L), color=color.white)
Rwall = box(pos = vec(2*L, 0, 0), size = vec(thick, L, L), color=color.white)
Bwall = box(pos = vec(0, -L/2, 0), size = vec(4*L, thick, L), color=color.white)
Twall = box(pos = vec(0, L/2, 0), size = vec(4*L, thick, L), color=color.white)
Zwall = box(pos = vec(0, 0, -L/2), size = vec(4*L, L, thick), color=color.white)

particles = [] #list of particles

N=4 #number of particles

#create particles
for i in range(N):
    if i==0:
        particle = sphere(pos = L/2*vec(rand.uniform(-2,-1),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.cyan, make_trail=True, retain=20, trail_radius=0.3*R)
    else:
        particle = sphere(pos = L/2*vec(rand.uniform(-2,-1),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.red)

    #initial velocity
    particle.v = s*hat(vec(1, rand.uniform(-1,1),rand.uniform(-1,1)))
    particle.p = m*particle.v

    particles.append(particle)

#check for collisions and rerandomize positions of particles that are collided
collided_pairs = check_collisions()
while len(collided_pairs)!=0:
    for pair in collided_pairs:
        particles[pair[0]].pos = L/2*vec(rand.uniform(-1,1),rand.uniform(-1,1),rand.uniform(-1,1))
        particles[pair[1]].pos = L/2*vec(rand.uniform(-1,1),rand.uniform(-1,1),rand.uniform(-1,1))
    collided_pairs = check_collisions()

#time
t = 0
dt = R/s/10
#evolution
scene.pause()

Nsteps = 1e4 #number of time steps for the loop

while t < Nsteps*dt:
    rate(500) #sets number of loops per second in order to slow down or speed up visualization

    #update position of each particle
    for particle in particles:
        particle.pos = particle.pos + particle.v*dt
   
    #handle collisions of particles with each other
    collided_pairs = check_collisions()
    for pair in collided_pairs:
        i = pair[0]
        j = pair[1]
        calc_collision(i,j)
    
    #handle collision with walls
    for particle in particles:
        if particle.pos.x > 2*L:
            particle.pos.x = -2*L
        elif particle.pos.x < -2*L:
            particle.pos.x = 2*L
        if particle.pos.y > L/2:
            particle.v.y = - abs(particle.v.y)
            particle.p = m*particle.v
        elif particle.pos.y < -L/2:
            particle.v.y = abs(particle.v.y)
            particle.p = m*particle.v

        if particle.pos.z > L/2:
            particle.v.z = - abs(particle.v.z)
            particle.p = m*particle.v
        elif particle.pos.z < -L/2:
            particle.v.z = abs(particle.v.z)
            particle.p = m*particle.v

    t = t + dt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Creation of the atoms in the system 
    there will be atoms that will be fixed objects in the model to simulate the the materials in the system which the electrons will be colliding with will its going through the system

In [ ]:
scene = canvas(title="N Particles in a Box")

#constants
m = 1.7e-27 #mass of atom in kg
R = 0.5e-10 #radius of atom in m
L = 40*R #length of box in m
thick = L/100 #thickness of box wall in m
k = 1.4e-23 #boltzmann constant
T = 300 #temp in K
s =  np.sqrt(2*3/2*k*T/m) #initial speed


Lwall = box(pos = vec(-2*L, 0, 0), size = vec(thick, L, L), color=color.white)
Rwall = box(pos = vec(2*L, 0, 0), size = vec(thick, L, L), color=color.white)
Bwall = box(pos = vec(0, -L/2, 0), size = vec(4*L, thick, L), color=color.white)
Twall = box(pos = vec(0, L/2, 0), size = vec(4*L, thick, L), color=color.white)
Zwall = box(pos = vec(0, 0, -L/2), size = vec(4*L, L, thick), color=color.white)

M = 6
#creating atoms in area
for x in range(M):
    for y in range(M):
        for z in range(M):
            if x % 2 == 0:
                atom = sphere(pos = vec(2*L*(x/(M-1)-.5),L/1.5*(y/(M-1)-.5),L*(z/(M-1)-.5)), radius = R, color = color.red)
            else:
                atom = sphere(pos = vec(1.8*L*(x/(M-1)-.5),L/2*(y/(M-1)-.5),L*(z/(M-1)-.5)), radius = R, color = color.red)

## Idea / Base on objects collioding with fixed objects
    this is the base model in a moving object colliding with a fixed object
    one of the change we would need to do from this compared to the whole project is creating a nessed loop since there would be multiple particles and atoms so we need the nested loop to check each one.
    the other change is the values since we have different R values.

In [ ]:
GlowScript 3.2 VPython
scene=canvas()
Rfixed = 1
Rball = 2*Rfixed
fixed = sphere(pos=vec(0,0,0), radius=Rfixed, color=color.yellow)
ball = sphere(pos=vec(0,4*Rball,0.5*(Rfixed+Rball)), radius=Rball, color=color.cyan, make_trail=True)
ball.v = vec(0,-3,0)

t = 0 
dt = 0.01

scene.pause()

while t < 5:
    rate(100)
    
    ball.pos = ball.pos + ball.v*dt
    
    r = ball.pos - fixed.pos
    if mag(r) < Rfixed+Rball:
        rhat = hat(r)
        vballrad = dot(ball.v,rhat)*rhat #rad comp of ball.v
        vballtan = ball.v - vballrad #tan comp of ball.v
        vballrad = - vballrad #reverse ball.rad due to collision
        ball.v = vballtan + vballrad #new ball.v after collision
        p= mag(r)
        print(p)

    t = t + dt

## Putting it together 
we are putting together all to create the model Idealy the electrons should be moving to the right to simulate the electrical circuit moving to the right. 

there is an issue with the collsion with the fixed object/atom



In [ ]:
scene = canvas(title="Drude Model electrons moving through the system")

#constants
m = 1.7e-27 #mass of atom in kg
R = 0.5e-10 #radius of atom in m
L = 40*R #length of box in m
thick = L/100 #thickness of box wall in m
k = 1.4e-23 #boltzmann constant
T = 300 #temp in K
s =  np.sqrt(2*3/2*k*T/m) #initial speed
Rfixed = 1.5*R


Lwall = box(pos = vec(-2*L, 0, 0), size = vec(thick, L, L), color=color.white)
Rwall = box(pos = vec(2*L, 0, 0), size = vec(thick, L, L), color=color.white)
Bwall = box(pos = vec(0, -L/2, 0), size = vec(4*L, thick, L), color=color.white)
Twall = box(pos = vec(0, L/2, 0), size = vec(4*L, thick, L), color=color.white)
Zwall = box(pos = vec(0, 0, -L/2), size = vec(4*L, L, thick), color=color.white)

particles = [] #list of particles

atoms = [] #list of atoms

N=20 #number of particles

M = 7 #number of atoms in area

#creating atoms in area
for x in range(M):
    for y in range(M):
        for z in range(M):
            if x % 2 == 0:
                atom = sphere(pos = vec(2*L*(x/(M-1)-.5),L/1.5*(y/(M-1)-.5),L*(z/(M-1)-.5)), radius = Rfixed, color = color.red)
            else:
                atom = sphere(pos = vec(1.8*L*(x/(M-1)-.5),L/2*(y/(M-1)-.5),L*(z/(M-1)-.5)), radius = Rfixed, color = color.red)

#create particles
for i in range(N):
    if i==0:
        particle = sphere(pos = L/2*vec(rand.uniform(-2,-1),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.cyan, make_trail=True, retain=20, trail_radius=0.3*R)
    else:
        particle = sphere(pos = L/2*vec(rand.uniform(-2,-1),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.red)

    #initial velocity
    particle.v = s*hat(vec(1, rand.uniform(-1,1),rand.uniform(-1,1)))
    particle.p = m*particle.v

    particles.append(particle)
    atoms.append(atom)

#check for collisions and rerandomize positions of particles that are collided
collided_pairs = check_collisions()
while len(collided_pairs)!=0:
    for pair in collided_pairs:
        particles[pair[0]].pos = L/2*vec(rand.uniform(-1,1),rand.uniform(-1,1),rand.uniform(-1,1))
        particles[pair[1]].pos = L/2*vec(rand.uniform(-1,1),rand.uniform(-1,1),rand.uniform(-1,1))
    collided_pairs = check_collisions()

#time
t = 0
dt = R/s/10
#evolution
scene.pause()

Nsteps = 1e4 #number of time steps for the loop

while t < Nsteps*dt:
    rate(500) #sets number of loops per second in order to slow down or speed up visualization

    #update position of each particle
    for particle in particles:
        particle.pos = particle.pos + particle.v*dt
   
    #update the collision of fixed object and particle  
    for particle in particles:
        for atom in atoms:
            r = particle.pos - atom.pos
            if mag(r) < Rfixed+R:
                rhat = hat(r)
                vparticlerad = dot(particle.v,rhat)*rhat #rad comp of ball.v
                vparticletan = particle.v - vparticlerad #tan comp of ball.v
                vparticlerad = - vparticlerad #reverse ball.rad due to collision
                particle.v = vparticletan + vparticlerad #new ball.v after collision
                particle.pos = particle.pos + particle.v*dt
            

    #handle collisions of particles with each other
    collided_pairs = check_collisions()
    for pair in collided_pairs:
        i = pair[0]
        j = pair[1]
        calc_collision(i,j)
    
    #handle collision with walls
    for particle in particles:
        if particle.pos.x > 2*L:
            particle.pos.x = -2*L
        elif particle.pos.x < -2*L:
            particle.pos.x = 2*L
        if particle.pos.y > L/2:
            particle.v.y = - abs(particle.v.y)
            particle.p = m*particle.v
        elif particle.pos.y < -L/2:
            particle.v.y = abs(particle.v.y)
            particle.p = m*particle.v

        if particle.pos.z > L/2:
            particle.v.z = - abs(particle.v.z)
            particle.p = m*particle.v
        elif particle.pos.z < -L/2:
            particle.v.z = abs(particle.v.z)
            particle.p = m*particle.v

    t = t + dt


<IPython.core.display.Javascript object>

1.25e-10
